<a href="https://colab.research.google.com/github/KrishnaRao1/NFL-passing-yards-model/blob/main/NFL_snack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip?

In [ ]:
import pymc as pm
import re
import arviz as az
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup, Comment
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Map full names to abbreviations
team_name_map = {
    "Arizona Cardinals": "ARI",
    "Atlanta Falcons": "ATL",
    "Baltimore Ravens": "BAL",
    "Buffalo Bills": "BUF",
    "Carolina Panthers": "CAR",
    "Chicago Bears": "CHI",
    "Cincinnati Bengals": "CIN",
    "Cleveland Browns": "CLE",
    "Dallas Cowboys": "DAL",
    "Denver Broncos": "DEN",
    "Detroit Lions": "DET",
    "Green Bay Packers": "GB",
    "Houston Texans": "HOU",
    "Indianapolis Colts": "IND",
    "Jacksonville Jaguars": "JAX",
    "Kansas City Chiefs": "KC",
    "Las Vegas Raiders": "LV",
    "Los Angeles Chargers": "LAC",
    "Los Angeles Rams": "LAR",
    "Miami Dolphins": "MIA",
    "Minnesota Vikings": "MIN",
    "New England Patriots": "NE",
    "New Orleans Saints": "NO",
    "New York Giants": "NYG",
    "New York Jets": "NYJ",
    "Philadelphia Eagles": "PHI",
    "Pittsburgh Steelers": "PIT",
    "San Francisco 49ers": "SF",
    "Seattle Seahawks": "SEA",
    "Tampa Bay Buccaneers": "TB",
    "Tennessee Titans": "TEN",
    "Washington Commanders": "WSH"
}

In [ ]:
# Drag and drop .csv files into datalab file

#passing_2024 = pd.read_csv("/content/2024 passing stats.csv")
#passing_2024 = passing_2024.drop(columns=['Awards','Player-additional'])

#scrimmage_2024 = pd.read_csv("/content/2024 scrimmage yards.csv")
#scrimmage_2024 = scrimmage_2024.drop(columns=['Awards', '-9999', 'Rk'])


schedule_2025 = pd.read_csv("/content/Team Schedule 2025.csv")
joshie = pd.read_csv("/content/josh_allen.csv")
maho = pd.read_csv("/content/maho_career.csv")
aaron = pd.read_csv("/content/AAron.csv")

passing_defense_2024 = pd.read_csv("/content/passing defense.csv")

allen_2024 = pd.read_csv("/content/allen_2024.csv")
maho_2024 = pd.read_csv("/content/maho_2024.csv")
rodgers_2024 = pd.read_csv("/content/rodgers_2024.csv")


In [ ]:
schedule_2025.columns = ['Team', 'wk 1', 'wk 2', 'wk 3', 'wk 4', 'wk 5', 'wk 6', 'wk 7', 'wk 8', 'wk 9', 'wk 10', 'wk 11', 'wk 12', 'wk 13', 'wk 14', 'wk 15', 'wk 16', 'wk 17', 'wk 18']
qb_data = {'Team': ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LAC', 'LAR', 'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SF', 'SEA', 'TB', 'TEN', 'WSH'],
           'QB': ['Kyler Murray', 'Michael Penix', 'Lamar Jackson', 'Josh Allen', 'Bryce Young', 'Caleb Williams', 'Joe Burrow', 'Joe Flacco', 'Dak Prescott', 'Bo Nix', 'Jared Goff', 'Jordan Love', 'C.J. Stroud', 'Daniel Jones', 'Trevor Lawrence', 'Patrick Mahomes', 'Justin Herbert', 'Matthew Stafford', 'Geno Smith', 'Tua Tagovailoa', 'JJ McCarthy', 'Drake Maye', 'Spencer Rattler', 'Russell Wilson', 'Justin Fields', 'Jalen Hurts', 'Aaron Rodgers', 'Brock Purdy', 'Sam Darnold', 'Baker Mayfield', 'Cam Ward', 'Jayden Daniels']}
qb = pd.DataFrame(qb_data)
qb_schedule = pd.merge(qb, schedule_2025, on='Team')



In [ ]:
selected_qbs = qb_schedule[qb_schedule['QB'].isin(['Josh Allen', 'Patrick Mahomes', 'Aaron Rodgers'])]
selected_qbs

,Team,QB,wk 1,wk 2,wk 3,wk 4,wk 5,wk 6,wk 7,wk 8,wk 9,wk 10,wk 11,wk 12,wk 13,wk 14,wk 15,wk 16,wk 17,wk 18
2,BUF,Josh Allen,BAL,@NYJ,MIA,NO,NE,@ATL,BYE,@CAR,KC,@MIA,TB,@HOU,@PIT,CIN,@NE,@CLE,PHI,NYJ
14,KC,Patrick Mahomes,@LAC,PHI,@NYG,BAL,@JAX,DET,LV,WSH,@BUF,BYE,@DEN,IND,@DAL,HOU,LAC,@TEN,DEN,@LV
25,PIT,Aaron Rodgers,@NYJ,SEA,@NE,MIN,BYE,CLE,@CIN,GB,IND,@LAC,CIN,@CHI,BUF,@BAL,MIA,@DET,@CLE,BAL


In [ ]:
joshie['Opponent'] = joshie['Team'].map(team_name_map)
maho['Opponent'] = maho['Team'].map(team_name_map)
aaron['Opponent'] = aaron['Team'].map(team_name_map)

In [ ]:
joshie['YDS_per_GP'] = joshie['Pass Yds'] / joshie['Games']
maho['YDS_per_GP'] = maho['Pass Yds'] / maho['Games']


In [ ]:
aaron

,Team,Games,Passing Yards,Yards per Game,Opponent
0,ARI,8,2384,298.000000,NaN
1,ATL,10,2890,289.000000,NaN
2,BAL,4,1098,274.500000,NaN
3,BUF,6,1654,275.666667,NaN
4,CAR,7,1987,283.857143,NaN
5,CHI,30,8456,281.866667,NaN
6,CIN,4,1124,281.000000,NaN
7,CLE,3,798,266.000000,NaN
8,DAL,11,3189,289.909091,NaN
9,DEN,4,1167,291.750000,NaN


In [ ]:
joshie = (joshie[['Opponent', 'YDS_per_GP']])
maho = (maho[['Team', 'YDS_per_GP']])
aaron = (aaron[['Team', 'Yards per game']])

KeyError: "['Yards per game'] not in index"

In [ ]:
# Drop all rows with any NA values
joshie = joshie.dropna()
maho = maho.dropna()

# Reset the index after dropping
joshie = joshie.reset_index(drop=True)
maho = maho.reset_index(drop=True)

In [ ]:
# Rename columns for clarity
joshie_clean = joshie.rename(columns={'YDS_per_GP': 'allen_avg', 'Opponent': 'Team'})
maho_clean  = maho.rename(columns={'YDS_per_GP': 'mahomes_avg'})

In [ ]:
# Merge on team
combined = joshie_clean.merge(maho_clean, on='Team', how='outer')
combined.head(10)

,Team,allen_avg,mahomes_avg
0,ARI,271.000000,305.000000
1,ATL,249.000000,217.000000
2,BAL,268.000000,217.800000
3,BUF,NaN,183.777778
4,CAR,237.000000,372.000000
5,CHI,298.000000,277.000000
6,CIN,247.666667,186.857143
7,CLE,229.000000,229.500000
8,DAL,293.500000,403.000000
9,DEN,261.250000,271.285714


In [ ]:
def reshape_schedule_per_qb(qb_schedule_df):
    """
    Returns a dictionary of schedules keyed by QB name.
    Each value is a long-format schedule dataframe for that QB.
    """
    schedule_long = qb_schedule_df.melt(
        id_vars=["Team", "QB"],
        var_name="Week",
        value_name="Opp_Team"
    )
    schedule_long["Week"] = schedule_long["Week"].str.extract(r"(\d+)").astype(int)
    schedule_long["projected_yards"] = None
    schedule_long["is_home"] = (~schedule_long["Opp_Team"].astype(str).str.startswith("@")).astype(int)
    schedule_long["Opp_Team"] = schedule_long["Opp_Team"].astype(str).str.replace("@", "", regex=False).str.strip()

    # Create separate table per QB
    qb_tables = {}
    for qb in schedule_long["QB"].unique():
        qb_tables[qb] = schedule_long[schedule_long["QB"] == qb].sort_values(by="Week").reset_index(drop=True)

    return qb_tables

In [ ]:
qb_schedules = reshape_schedule_per_qb(selected_qbs)

In [ ]:
# Defensive yards allowed
defense_yards = passing_defense_2024[['Tm', 'Yds']].copy()
defense_yards.rename(columns={'Tm': 'Opp_Team', 'Yds': 'Def_YPG'}, inplace=True)
defense_yards = defense_yards.iloc[:-3].reset_index(drop=True)


In [ ]:
# Convert full names -> abbreviations
# Use the 'Opp_Team' column for mapping as it contains the team names in defense_yards
defense_yards['Opp_Team_abbr'] = defense_yards['Opp_Team'].map(team_name_map)

In [ ]:
# Keep only abbreviation + Def_YPG
defense_yards = defense_yards[['Opp_Team_abbr','Def_YPG']]
defense_yards.rename(columns={'Opp_Team_abbr':'Opp_Team'}, inplace=True)
defense_yards.head(3)

,Opp_Team,Def_YPG
0,PHI,2961.0
1,TEN,3014.0
2,WSH,3221.0


In [ ]:
# Assume qb_schedules is the dict from reshape_schedule_per_qb()
# defense_yards has columns: Opp_Team, Def_YPG

for qb, df in qb_schedules.items():
    # Merge defensive stats
    df = df.merge(defense_yards, on='Opp_Team', how='left')

    # Remove BYE weeks
    df = df.query("Opp_Team != 'BYE'").copy()

    # Convert Def_YPG to per-game
    df['Def_YPG'] = df['Def_YPG'] / 17

    # Update dictionary
    qb_schedules[qb] = df.reset_index(drop=True)

In [ ]:
josh_schedule = qb_schedules['Josh Allen']
maho_schedule = qb_schedules['Patrick Mahomes']


In [ ]:
# Rename offensive averages for merging
joshie_clean = joshie.rename(columns={'Opponent': 'Opp_Team', 'YDS_per_GP': 'allen_avg'})
maho_clean  = maho.rename(columns={'Team': 'Opp_Team', 'YDS_per_GP': 'mahomes_avg'})

# Merge into each QB's table individually
for qb, df in qb_schedules.items():
    if qb == 'Josh Allen':
        df = df.merge(joshie_clean, on='Opp_Team', how='left')
    elif qb == 'Patrick Mahomes':
        df = df.merge(maho_clean, on='Opp_Team', how='left')

    qb_schedules[qb] = df.reset_index(drop=True)




In [ ]:
# Create a tab for each QB
tabs = widgets.Tab()

children = []

# Use the separate QB schedules dictionary

for i, (qb, qb_df) in enumerate(qb_schedules.items()):
    # Select relevant columns to display
    if qb == 'Josh Allen':
        display_cols = ["Week", "Team", "Opp_Team", "is_home", "projected_yards", "Def_YPG", "allen_avg"]
    elif qb == 'Patrick Mahomes':
        display_cols = ["Week", "Team", "Opp_Team", "is_home", "projected_yards", "Def_YPG", "mahomes_avg"]

    output = widgets.Output()
    with output:
        display(qb_df[display_cols])
    children.append(output)

tabs.children = children

# Set tab titles to QB names
for i, qb in enumerate(qb_schedules.keys()):
    tabs.set_title(i, qb)

display(tabs)

In [ ]:
# Step 1: check QB schedule tables
for qb, df in qb_schedules.items():
    print(qb)
    print(df.head())
    print(df.columns)
    print(df[['Def_YPG', 'allen_avg' if qb=='Josh Allen' else 'mahomes_avg']].isna().sum())

Josh Allen
  Team          QB  Week Opp_Team projected_yards  is_home     Def_YPG  \
0  BUF  Josh Allen     1      BAL            None        1  244.117647   
1  BUF  Josh Allen     2      NYJ            None        0  192.647059   
2  BUF  Josh Allen     3      MIA            None        1  210.705882   
3  BUF  Josh Allen     4       NO            None        1  238.529412   
4  BUF  Josh Allen     5       NE            None        1  211.529412   

    allen_avg  
0  268.000000  
1  272.461538  
2  285.800000  
3  254.000000  
4  269.500000  
Index(['Team', 'QB', 'Week', 'Opp_Team', 'projected_yards', 'is_home',
       'Def_YPG', 'allen_avg'],
      dtype='object')
Def_YPG      0
allen_avg    0
dtype: int64
Patrick Mahomes
  Team               QB  Week Opp_Team projected_yards  is_home     Def_YPG  \
0   KC  Patrick Mahomes     1      LAC            None        0  206.882353   
1   KC  Patrick Mahomes     2      PHI            None        1  174.176471   
2   KC  Patrick Mahomes    

In [ ]:
#MODEL

In [ ]:


# Step 2: rebuild training_data safely
training_data = {}

for qb, df in qb_schedules.items():
    target_col = 'allen_avg' if qb == 'Josh Allen' else 'mahomes_avg'

    # Only drop rows where both predictor and target are missing
    df_train = df.dropna(subset=['Def_YPG', target_col])

    if df_train.empty:
        print(f"Warning: No training data for {qb}")
        continue

    X = df_train[['Def_YPG']].values.astype(float)
    y = df_train[target_col].values.astype(float)

    X_mean = X.mean(axis=0)
    X_std = X.std(axis=0)
    X_scaled = (X - X_mean) / X_std

    training_data[qb] = {'X': X_scaled, 'y': y, 'X_mean': X_mean, 'X_std': X_std}

#print("Training data keys:", training_data.keys())

In [ ]:
# Dictionary to hold traces and models per QB
bayesian_models = {}

for qb, data in training_data.items():
    X_scaled = data['X'].flatten()  # predictor
    y = data['y']                    # target

    with pm.Model() as model:
        # Priors
        intercept = pm.Normal("intercept", mu=0, sigma=100)
        beta = pm.Normal("beta", mu=0, sigma=50)
        sigma = pm.HalfNormal("sigma", sigma=50)

        # Linear predictor
        mu = intercept + beta * X_scaled

        # Likelihood
        pm.Normal("obs", mu=mu, sigma=sigma, observed=y)

        # Sample posterior
        trace = pm.sample(2000, tune=1000, target_accept=0.9, return_inferencedata=True)

    # Save model and trace for this QB
    bayesian_models[qb] = {
        'model': model,
        'trace': trace
    }

Output()

Output()

In [ ]:
for qb, data in bayesian_models.items():
    df = qb_schedules[qb]  # QB's schedule
    X_mean = training_data[qb]['X_mean']
    X_std = training_data[qb]['X_std']

    # Scale all weeks' defensive data
    X_all = df[['Def_YPG']].values.astype(float)
    X_all_scaled = (X_all - X_mean) / X_std

    # Extract posterior samples
    trace = data['trace']
    beta_samps = trace.posterior['beta'].stack(draws=("chain","draw")).values
    intercept_samps = trace.posterior['intercept'].stack(draws=("chain","draw")).values

    # Generate predictions: samples x games
    y_samps = intercept_samps[:, None] + beta_samps[:, None] * X_all_scaled.T

    # Take posterior mean as week-by-week projection
    projected_yards = y_samps.mean(axis=0)

    # Optional: clip to reasonable range
    projected_yards = np.clip(projected_yards, 100, 500)

    # Assign to dataframe
    df['projected_yards'] = projected_yards
    qb_schedules[qb] = df

In [ ]:
qb_schedules['Josh Allen'][['Week','Opp_Team','projected_yards']]

,Week,Opp_Team,projected_yards
0,1,BAL,255.159195
1,2,NYJ,260.000620
2,3,MIA,258.301971
3,4,NO,255.684835
4,5,NE,258.224509
5,6,ATL,257.001703
6,8,CAR,256.985104
7,9,KC,257.538410
8,10,MIA,258.301971
9,11,TB,255.175794


In [ ]:
# Create a tab for each QB
tabs = widgets.Tab()
children = []

for i, (qb, df) in enumerate(qb_schedules.items()):
    # Columns to display
    display_cols = ["Week", "Team", "Opp_Team", "is_home", "projected_yards"]

    # Make a copy so rounding doesn't affect the original df
    df_display = df[display_cols].copy()

    # Round projected_yards
    if "projected_yards" in df_display.columns:
        df_display["projected_yards"] = df_display["projected_yards"].round().astype(int)

    output = widgets.Output()
    with output:
        display(df_display)
    children.append(output)

tabs.children = children

# Set tab titles to QB names
for i, qb in enumerate(qb_schedules.keys()):
    tabs.set_title(i, qb)

display(tabs)

In [ ]:
#CONFIDENCE INTERVALS


ci_tables = {}

for qb, data in bayesian_models.items():
    df = qb_schedules[qb]  # for Week numbers
    trace = data['trace']

    # Extract posterior samples
    beta_samps = trace.posterior['beta'].stack(draws=("chain","draw")).values
    intercept_samps = trace.posterior['intercept'].stack(draws=("chain","draw")).values

    # Scale defensive data
    X_mean = training_data[qb]['X_mean']
    X_std = training_data[qb]['X_std']
    X_all = df[['Def_YPG']].values.astype(float)
    X_all_scaled = (X_all - X_mean) / X_std

    # Generate posterior predictive samples: samples x games
    y_samps = intercept_samps[:, None] + beta_samps[:, None] * X_all_scaled.T

    # Compute 90% credible interval
    ci_low = np.percentile(y_samps, 5, axis=0)
    ci_high = np.percentile(y_samps, 95, axis=0)

    # Build a table with Week and CI
    ci_df = pd.DataFrame({
        'Week': df['Week'],
        'Opp_Team': df['Opp_Team'],
        'ci_low': ci_low.round().astype(int),
        'ci_high': ci_high.round().astype(int)
    })

    ci_tables[qb] = ci_df



In [ ]:
import ipywidgets as widgets

# Create a tab for each QB
tabs = widgets.Tab()
children = []

for i, (qb, ci_df) in enumerate(ci_tables.items()):  # use ci_tables instead of qb_schedules
    # Columns to display
    display_cols = ["Week", "Opp_Team", "ci_low", "ci_high"]

    # Make a copy so rounding doesn't affect original df
    df_display = ci_df[display_cols].copy()

    output = widgets.Output()
    with output:
        display(df_display)
    children.append(output)

tabs.children = children

# Set tab titles to QB names
for i, qb in enumerate(ci_tables.keys()):
    tabs.set_title(i, qb)

display(tabs)
